## Question 1

In [13]:
# Local Search (Best Improvement) for 0-1 Knapsack
# Author: Orkhan Khankishiyev
# Based on: Base code by Charles Nicholson (Do not change random instance)

import copy
from random import Random
import numpy as np

# --- Problem Setup (Do Not Change) ---
seed = 51132023
myPRNG = Random(seed)
n = 150

# Random values and weights
value = [round(myPRNG.triangular(5, 1000, 200), 1) for _ in range(n)]
weights = [round(myPRNG.triangular(10, 200, 60), 1) for _ in range(n)]

maxWeight = 2500  # Knapsack capacity

# --- Evaluation Function with Penalization ---
def evaluate(x):
    a = np.array(x)
    totalValue = np.dot(a, value)
    totalWeight = np.dot(a, weights)
    if totalWeight > maxWeight:
        return [0, totalWeight]  # penalize infeasible
    return [totalValue, totalWeight]

# --- Initial Solution: All zeros (empty knapsack) ---
def initial_solution():
    return [0 for _ in range(n)]

# --- 1-Flip Neighborhood ---
def neighborhood(x):
    nbrhood = []
    for i in range(n):
        neighbor = x[:]
        neighbor[i] = 1 - neighbor[i]
        nbrhood.append(neighbor)
    return nbrhood

# --- Results Summary Helper ---
def summarize_results(name, iterations, x_best, f_best):
    print(f"{name}")
    print(f"Iterations: {iterations}")
    print(f"# Items Selected: {int(np.sum(x_best))}")
    print(f"Weight: {round(f_best[1], 2)}")
    print(f"Objective: {round(f_best[0], 2)}")
    print("------")

# --- Local Search Logic (Best Improvement) ---
solutionsChecked = 0
x_curr = initial_solution()
x_best = x_curr[:]
f_curr = evaluate(x_curr)
f_best = f_curr[:]

done = False
while not done:
    Neighborhood = neighborhood(x_curr)
    improved = False

    for s in Neighborhood:
        solutionsChecked += 1
        score = evaluate(s)
        if score[0] > f_best[0]:
            x_best = s[:]
            f_best = score[:]
            improved = True

    if improved:
        x_curr = x_best[:]
        f_curr = f_best[:]
    else:
        done = True

# --- Print Final Results ---
summarize_results("Local Search (Best Improvement)", solutionsChecked, x_best, f_best)


Local Search (Best Improvement)
Iterations: 4350
# Items Selected: 28
Weight: 2491.5
Objective: 21408.7
------


In [27]:
# Local Search with Random Restarts for 0-1 Knapsack

import copy
from random import Random
import numpy as np

# --- Problem Setup (Do Not Modify) ---
seed = 51132023
myPRNG = Random(seed)
n = 150

value = [round(myPRNG.triangular(5, 1000, 200), 1) for _ in range(n)]
weights = [round(myPRNG.triangular(10, 200, 60), 1) for _ in range(n)]
maxWeight = 2500

# --- Evaluation Function with Penalization ---
def evaluate(x):
    a = np.array(x)
    totalValue = np.dot(a, value)
    totalWeight = np.dot(a, weights)
    if totalWeight > maxWeight:
        return [0, totalWeight]
    return [totalValue, totalWeight]

# --- Neighborhood Function (1-flip) ---
def neighborhood(x):
    nbrhood = []
    for i in range(n):
        neighbor = x[:]
        neighbor[i] = 1 - neighbor[i]
        nbrhood.append(neighbor)
    return nbrhood

# --- Random Feasible Initial Solution ---
def random_feasible_solution():
    x = [0 for _ in range(n)]
    indices = list(range(n))
    myPRNG.shuffle(indices)

    total_weight = 0
    for i in indices:
        if total_weight + weights[i] <= maxWeight:
            x[i] = 1
            total_weight += weights[i]
    return x

# --- Local Search (Best Improvement) from Initial Solution ---
def best_improvement(x_start):
    global solutionsChecked
    x_curr = x_start[:]
    f_curr = evaluate(x_curr)
    x_best = x_curr[:]
    f_best = f_curr[:]

    done = False
    while not done:
        Neighborhood = neighborhood(x_curr)
        improved = False

        for s in Neighborhood:
            solutionsChecked += 1
            score = evaluate(s)
            if score[0] > f_best[0]:
                x_best = s[:]
                f_best = score[:]
                improved = True

        if improved:
            x_curr = x_best[:]
            f_curr = f_best[:]
        else:
            done = True

    return x_best, f_best

# --- Results Summary ---
def summarize_results(name, iterations, x_best, f_best):
    print(f"{name}")
    print(f"Iterations: {iterations}")
    print(f"# Items Selected: {int(np.sum(x_best))}")
    print(f"Weight: {round(f_best[1], 2)}")
    print(f"Objective: {round(f_best[0], 2)}")
    print("------")

# --- Random Restarts Logic ---
solutionsChecked = 0
num_restarts = 25

overall_best_value = 0
overall_best_solution = None
overall_best_eval = None

for _ in range(num_restarts):
    x_start = random_feasible_solution()
    x_local, f_local = best_improvement(x_start)

    if f_local[0] > overall_best_value:
        overall_best_value = f_local[0]
        overall_best_solution = x_local[:]
        overall_best_eval = f_local[:]

summarize_results("Local Search with Random Restarts", solutionsChecked, overall_best_solution, overall_best_eval)


Local Search with Random Restarts
Iterations: 3750
# Items Selected: 32
Weight: 2499.1
Objective: 15341.1
------


In [29]:
# Local Search with Random Walk for 0-1 Knapsack

import copy
from random import Random
import numpy as np

# --- Problem Setup (Do Not Modify) ---
seed = 51132023
myPRNG = Random(seed)
n = 150

value = [round(myPRNG.triangular(5, 1000, 200), 1) for _ in range(n)]
weights = [round(myPRNG.triangular(10, 200, 60), 1) for _ in range(n)]
maxWeight = 2500

# --- Evaluation Function with Penalization ---
def evaluate(x):
    a = np.array(x)
    totalValue = np.dot(a, value)
    totalWeight = np.dot(a, weights)
    if totalWeight > maxWeight:
        return [0, totalWeight]
    return [totalValue, totalWeight]

# --- 1-Flip Neighborhood ---
def neighborhood(x):
    nbrhood = []
    for i in range(n):
        neighbor = x[:]
        neighbor[i] = 1 - neighbor[i]
        nbrhood.append(neighbor)
    return nbrhood

# --- Initial Solution (Feasible Random) ---
def random_feasible_solution():
    x = [0 for _ in range(n)]
    indices = list(range(n))
    myPRNG.shuffle(indices)

    total_weight = 0
    for i in indices:
        if total_weight + weights[i] <= maxWeight:
            x[i] = 1
            total_weight += weights[i]
    return x

# --- Results Summary ---
def summarize_results(name, iterations, x_best, f_best):
    print(f"{name}")
    print(f"Iterations: {iterations}")
    print(f"# Items Selected: {int(np.sum(x_best))}")
    print(f"Weight: {round(f_best[1], 2)}")
    print(f"Objective: {round(f_best[0], 2)}")
    print("------")

# --- Local Search with Random Walk ---
solutionsChecked = 0
p = 0.25  # probability of random move
iterations_limit = 5000

x_curr = random_feasible_solution()
f_curr = evaluate(x_curr)
x_best = x_curr[:]
f_best = f_curr[:]

for _ in range(iterations_limit):
    solutionsChecked += 1
    neighbors = neighborhood(x_curr)

    if myPRNG.random() < p:
        # Randomly choose a neighbor
        candidate = neighbors[myPRNG.randint(0, len(neighbors) - 1)]
    else:
        # Best improvement
        candidate = x_curr
        for s in neighbors:
            if evaluate(s)[0] > evaluate(candidate)[0]:
                candidate = s

    f_candidate = evaluate(candidate)

    if f_candidate[0] > f_best[0]:
        x_best = candidate[:]
        f_best = f_candidate[:]

    x_curr = candidate[:]

summarize_results("Local Search with Random Walk", solutionsChecked, x_best, f_best)


Local Search with Random Walk
Iterations: 5000
# Items Selected: 36
Weight: 2480.8
Objective: 20676.4
------


## Question 6

In [25]:
# Stochastic Hill Climbing – Q6
# Based on base code by Charles Nicholson
# Modified by Orkhan Khankishiyev

import copy
from random import Random
import numpy as np
import math

# ---------------------------- Base Problem Setup (DO NOT MODIFY) ----------------------------

seed = 51132023
myPRNG = Random(seed)
n = 150

value = []
for i in range(0, n):
    value.append(round(myPRNG.triangular(5, 1000, 200), 1))

weights = []
for i in range(0, n):
    weights.append(round(myPRNG.triangular(10, 200, 60), 1))

maxWeight = 2500

# ---------------------------- Evaluation Function ----------------------------

def evaluate(x):
    a = np.array(x)
    b = np.array(value)
    c = np.array(weights)

    totalValue = np.dot(a, b)
    totalWeight = np.dot(a, c)

    if totalWeight > maxWeight:
        return [0, totalWeight]  # penalize infeasible
    return [totalValue, totalWeight]

# ---------------------------- 1-Flip Neighborhood ----------------------------

def neighborhood(x):
    nbrhood = []
    for i in range(n):
        neighbor = x[:]
        neighbor[i] = 1 - neighbor[i]
        nbrhood.append(neighbor)
    return nbrhood

# ---------------------------- Random Feasible Initial Solution ----------------------------

def random_feasible_solution():
    x = [0 for _ in range(n)]
    indices = list(range(n))
    myPRNG.shuffle(indices)
    total_weight = 0
    for i in indices:
        if total_weight + weights[i] <= maxWeight:
            x[i] = 1
            total_weight += weights[i]
    return x

# ---------------------------- Softmax Probability Function ----------------------------

def softmax(scores, alpha=0.01):
    exp_scores = [math.exp(alpha * s) for s in scores]
    total = sum(exp_scores)
    return [x / total for x in exp_scores]

# ---------------------------- Stochastic Hill Climbing ----------------------------

solutionsChecked = 0
iterations = 5000

x_curr = random_feasible_solution()
f_curr = evaluate(x_curr)
x_best = x_curr[:]
f_best = f_curr[:]

for _ in range(iterations):
    Neighborhood = neighborhood(x_curr)
    feasible_neighbors = []
    neighbor_scores = []

    for s in Neighborhood:
        score = evaluate(s)
        solutionsChecked += 1
        if score[1] <= maxWeight:
            feasible_neighbors.append(s)
            neighbor_scores.append(score[0])

    if not feasible_neighbors:
        continue

    probabilities = softmax(neighbor_scores, alpha=0.01)
    chosen_index = np.random.choice(len(feasible_neighbors), p=probabilities)
    x_curr = feasible_neighbors[chosen_index]
    f_curr = evaluate(x_curr)

    if f_curr[0] > f_best[0]:
        x_best = x_curr[:]
        f_best = f_curr[:]

# ---------------------------- Final Output in Report Format ----------------------------

print("\nSummary for Report:")
print(f"{'Algorithm':<30}{'Iterations':<12}{'# Items Selected':<18}{'Weight':<10}{'Objective'}")
print(f"{'Stochastic Hill Climbing':<30}{solutionsChecked:<12}{int(np.sum(x_best)):<18}{round(f_best[1],2):<10}{round(f_best[0],2)}")



Summary for Report:
Algorithm                     Iterations  # Items Selected  Weight    Objective
Stochastic Hill Climbing      750000      43                2497.3    25766.6
